In [3]:
import yaml
import alfworld
import alfworld.agents.environment
with open('tests/assets/base_config.yaml') as reader:
    config = yaml.safe_load(reader)
    
print('hello wolrd')
split = "eval_out_of_distribution"

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]    
    return ob

hello wolrd
Initializing AlfredTWEnv...
Checking for solvable games...


100%|██████████| 341/341 [00:00<00:00, 1035.96it/s]

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [2]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_PROMPTS_EXAMPLE
d = REACT_ALFWORLD_PROMPTS_EXAMPLE

In [3]:

def alfworld_run(prompt, to_print=True, ob=''):
    init_prompt = prompt + ob + '\n>'
    prompt = ''
    if to_print:
        print(ob)
    for i in range(1, 50):
        action = llm(init_prompt + prompt, stop=['\n']).strip()
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        print(observation, reward, done)
        if action.startswith('think:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {action}\nObs {i}: {observation}')
        prompt += f' {action}\n{observation}\n>'
        if done:
            return reward
    return 0

In [4]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}



ob, info = env.reset()
ob = '\n'.join(ob[0].split('\n\n')[1:])
name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])
print(name)
for i, (k, v) in enumerate(prefixes.items()):
    if name.startswith(k):
        prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0'] + '\nHere is the task.\n'
        break



pick_cool_then_place_in_recep-Tomato-None-Microwave-10/trial_T20190909_102644_926781


In [5]:
k


'pick_cool_then_place'

In [6]:
ob

'You are in the middle of a room. Looking quickly around you, you see a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: put a cool tomato in microwave.'

In [3]:
from langchain_community.chat_models import ChatOpenAI


llm = ChatOpenAI(openai_api_key=)

In [4]:
from discussion_agents.cog.agent.react import ReActAgent


test_run = ReActAgent(llm = llm)

In [9]:
v

'cool'

In [10]:
examples = prompt

In [11]:
examples

'Interact with a household to solve a task. Here are two examples.\nYou are in the middle of a room. Looking quickly around you, you see a cabinet 9, a cabinet 8, a cabinet 7, a cabinet 6, a cabinet 5, a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a coffeemachine 1, a countertop 3, a countertop 2, a countertop 1, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a fridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin 1, a stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster 1.\nYour task is to: put a cool mug in shelf.\n> think: To solve the task, I need to find and take a mug, then cool it with fridge, then put it in shelf.\nOK.\n> think: First I need to find a mug. A mug is more likely to appear in countertop (1-3), coffeemachine (1), cabinet (1-9), shelf (1-3), drawer (1-9). I can check one by one, starting with countertop 1.\nOK.\n> go to countertop 1\nOn the c

In [12]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_INSTRUCTION

In [13]:

test_run.set_Alfworld()
observation = None
done = False
for i in range(0,50):
    ans = test_run.step(question=ob, examples=examples , prompt_template=REACT_ALFWORLD_INSTRUCTION , env_output=observation)
    print('output of agent step',ans)
    ans = ans[0].strip()
    observation, reward, done, info = env.step([ans])
    observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
    print(observation, reward, done)
    if ans.startswith('think:'):
        observation = 'OK.'
    if True:
        print(f'Act {i}: {ans}\nObs {i}: {observation}')
    if done:
        test_run.reset()
        break


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


output of agent step ['think: To solve the task, I need to find and take a tomato, then cool it with the fridge, and finally put it in/on the microwave.']
Nothing happens. False False
Act 0: think: To solve the task, I need to find and take a tomato, then cool it with the fridge, and finally put it in/on the microwave.
Obs 0: OK.
output of agent step ['think: First, I need to find a tomato. A tomato is more likely to appear in the fridge (1), countertop (1-3), shelf (1-3), cabinet (1-6), drawer (1-3). I can start by checking the fridge (1).']
Nothing happens. False False
Act 1: think: First, I need to find a tomato. A tomato is more likely to appear in the fridge (1), countertop (1-3), shelf (1-3), cabinet (1-6), drawer (1-3). I can start by checking the fridge (1).
Obs 1: OK.
output of agent step ['go to fridge 1']
The fridge 1 is closed. False False
Act 2: go to fridge 1
Obs 2: The fridge 1 is closed.
output of agent step ['open fridge 1']
You open the fridge 1. The fridge 1 is open.

In [21]:
test_run.is_think

True

In [1]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES, REACT_INSTRUCTION_HOTPOTQA

In [2]:
examples = REACT_WEBTHINK_SIMPLE6_FEWSHOT_EXAMPLES
instruction = REACT_INSTRUCTION_HOTPOTQA
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

In [9]:
for i in range(50):
    answer = test_run.step(question=question , examples=examples , prompt_template=instruction)
    print(answer[0][0])
    print(answer[0][1])
    print(answer[0][2])
    if answer[1]:
        break

I need to search for the best kick boxer in the world who has been involved in controversies and crimes of violence.
Search[best kick boxer in the world controversies]
Observation 1: Oleksandr Oleksandrovych Usyk (Ukrainian: Олекса́ндр Олекса́ндрович У́сик; born 17 January 1987) is a Ukrainian professional boxer. He has held multiple world championships in two weight classes, including the unified heavyweight titles since 2021, and the Ring magazine title since 2022. He has also held the International Boxing Organization (IBO) title since 2021. Previously, he held the undisputed cruiserweight championship from 2018 to 2019, being the first boxer in that division to hold all four major world titles. Usyk is widely regarded as one of the greatest Ukrainian boxers of all time and one of the greatest cruiserweights of all time, alongside Evander Holyfield. With his victory over Anthony Joshua in 2021, Usyk became one of only three boxers to have unified the cruiserweight world titles and b